We will use a custom callback that terminates the training process as soon as the validation accuracy reaches 75%.

In [1]:
!wget https://raw.githubusercontent.com/Alireza-Akhavan/SRU-deeplearning-workshop/master/dataset.py
!mkdir dataset
!wget https://github.com/Alireza-Akhavan/SRU-deeplearning-workshop/raw/master/dataset/Data_hoda_full.mat -P dataset

--2025-07-08 19:06:16--  https://raw.githubusercontent.com/Alireza-Akhavan/SRU-deeplearning-workshop/master/dataset.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 917 [text/plain]
Saving to: ‘dataset.py’

dataset.py          100%[===================>]     917  --.-KB/s    in 0s      

2025-07-08 19:06:16 (61.9 MB/s) - ‘dataset.py’ saved [917/917]

--2025-07-08 19:06:16--  https://github.com/Alireza-Akhavan/SRU-deeplearning-workshop/raw/master/dataset/Data_hoda_full.mat
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/Alireza-Akhavan/deeplearning-tensorflow2-notebooks/raw/master/dataset/Data_hoda

In [2]:
# 1. Import libraries and modules
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
import numpy as np
from dataset import load_hoda
import matplotlib.pyplot as plt

In [3]:

# Load pre-shuffled HODA data into train and test sets
X_train, y_train, X_test, y_test = load_hoda(
                                            training_sample_size=3500,
                                            test_sample_size=400,size=28)

In [4]:
# Preprocess input data
'''standard our data values to the range [-1, 1]'''
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = (X_train - 127.5) / 127.5
X_test = (X_test - 127.5) / 127.5

In [5]:
# Reshape to original image shape (n x 784)  ==> (n x 28 x 28 x 1)
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)

In [6]:
# 5. Define model architecture
model = Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
import tensorflow as tf
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, batch, logs=None):
        if logs['val_sparse_categorical_accuracy'] > 0.75:
            print()
            print('Accuracy reached 75 %!')
            self.model.stop_training=True

stopTrain = myCallback()

In [8]:
# 6. Compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['sparse_categorical_accuracy'])


# 7. Fit model on training data
history = model.fit(X_train, y_train,
          epochs=200, batch_size=256, validation_split=0.2,
              callbacks=[stopTrain])


Epoch 1/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 14s 529ms/step - loss: 2.3370 - sparse_categorical_accuracy: 0.3393 - val_loss: 2.0364 - val_sparse_categorical_accuracy: 0.6729
Epoch 2/200
 7/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.7123 - sparse_categorical_accuracy: 0.7779 
Accuracy reached 75 %!
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - loss: 0.6811 - sparse_categorical_accuracy: 0.7872 - val_loss: 1.8338 - val_sparse_categorical_accuracy: 0.7671
